<a href="https://colab.research.google.com/github/Kiritiaajd/brain-stroke-prediction/blob/main/brain_stroke_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

def plot_image_size_distribution(folder_path):
    sizes = []
    for file_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, file_name)
        with Image.open(img_path) as img:
            sizes.append(img.size)

    width, height = zip(*sizes)
    plt.figure(figsize=(12, 6))
    plt.hist(width, bins=30, alpha=0.5, label='Width')
    plt.hist(height, bins=30, alpha=0.5, label='Height')
    plt.xlabel('Size (pixels)')
    plt.ylabel('Frequency')
    plt.title('Image Size Distribution')
    plt.legend(loc='upper right')
    plt.show()

# Example usage:
plot_image_size_distribution('/content/dataset/Brain_Stroke_CT-SCAN_image/Train/hemorrhagic')


NameError: name 'os' is not defined

In [ ]:
from skimage import color
from skimage import exposure

def plot_image_quality(folder_path):
    brightness = []
    contrast = []
    for file_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, file_name)
        with Image.open(img_path) as img:
            # Check if the image is already grayscale
            if img.mode == 'L':
                img_gray = np.array(img)
            else:
                img_gray = color.rgb2gray(np.array(img))

            brightness.append(np.mean(img_gray))
            p2, p98 = np.percentile(img_gray, (2, 98))
            contrast.append(p98 - p2)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.hist(brightness, bins=30, color='blue', alpha=0.7)
    plt.xlabel('Brightness')
    plt.ylabel('Frequency')
    plt.title('Brightness Distribution')

    plt.subplot(1, 2, 2)
    plt.hist(contrast, bins=30, color='red', alpha=0.7)
    plt.xlabel('Contrast')
    plt.ylabel('Frequency')
    plt.title('Contrast Distribution')

    plt.tight_layout()
    plt.show()

# Example usage:
plot_image_quality('/content/dataset/Brain_Stroke_CT-SCAN_image/Train/hemorrhagic')

: 

: 

: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import os


: 

: 

: 

In [ ]:
class BrainStrokeDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        for label in ['hemorrhagic', 'ischaemic']:  # Ensure correct label names
            folder = os.path.join(root_dir, label)
            if os.path.exists(folder):  # Check if folder exists
                for file_name in os.listdir(folder):
                    if file_name.endswith('.jpg'):
                        self.image_paths.append(os.path.join(folder, file_name))
                        self.labels.append(0 if label == 'hemorrhagic' else 1)  # Example: 0 for hemorrhagic, 1 for ischaemic
            else:
                print(f"Directory does not exist: {folder}")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

: 

: 

: 

In [ ]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


: 

: 

: 

In [ ]:
import os

def check_directories(root_dir):
    for subfolder in os.listdir(root_dir):
        subfolder_path = os.path.join(root_dir, subfolder)
        if os.path.isdir(subfolder_path):
            print(f"Contents of {subfolder}:")
            print(os.listdir(subfolder_path))
        else:
            print(f"{subfolder_path} is not a directory")

# Verify the training directory
check_directories('/content/dataset/Brain_Stroke_CT-SCAN_image/Train')
# Verify the validation directory
check_directories('/content/dataset/Brain_Stroke_CT-SCAN_image/Validation')


: 

: 

: 

In [ ]:
train_dataset = BrainStrokeDataset(root_dir='/content/dataset/Brain_Stroke_CT-SCAN_image/Train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

valid_dataset = BrainStrokeDataset(root_dir='/content/dataset/Brain_Stroke_CT-SCAN_image/Validation', transform=transform)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)


: 

: 

: 

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 32 * 32, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


: 

: 

: 

In [ ]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

    print("Training complete")

# Train the model
train_model(model, train_loader, criterion, optimizer)


: 

: 

: 

In [ ]:
def evaluate_model(model, valid_loader, criterion):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    running_loss = 0.0
    correct_preds = 0
    total_preds = 0

    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct_preds += torch.sum(preds == labels.data)
            total_preds += labels.size(0)

    epoch_loss = running_loss / len(valid_loader.dataset)
    accuracy = correct_preds.double() / total_preds
    print(f"Validation Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}")

# Evaluate the model
evaluate_model(model, valid_loader, criterion)


: 

: 

: 

In [ ]:
# Save the model
torch.save(model.state_dict(), 'cnn_model.pth')

# Load the model
model = CNN()
model.load_state_dict(torch.load('cnn_model.pth'))
model.eval()


: 

: 

: 